In [4]:
from PyBTLS.py_module.bridge.bridge import Bridge
from PyBTLS.py_module.bridge.flow import LaneFlow, BridgeFlow
from PyBTLS.py_module.parallel.wrapper import BTLSWrapper
from PyBTLS.py_module.traffic.constant_vehicle import ConstantVehicle
from PyBTLS.py_module.traffic.garage import Garage, Kernels
from PyBTLS.py_module.traffic.PyBtlsDefaultTraffic import Auxerre #OLD CODE, BUT NO REPLACEMENT YET
import numpy as np

In [5]:
# Define the bridge
# This is similar to PyCBA syntax
L = [25.0] #m
R = [-1, 0, -1, 0] #Simply supported
bridge = Bridge(L=L, R=R, n_lane=1)

# Define the Load Effects to analyse
# Adding load effects can be done in two ways
# The first way is to use BTLS built-in influence line
# e.g., this is Influence Line 1, which computes moment at midspan
# See BTLS manual for full list of built-in load effects
bridge.add_load_effect(load_effect = 1) 
# Another way to define load effects is by specifiying:
#   load_effect ('M', 'V', or 'R' for moment, shear or reaction)
#   poi         (The point where the load effect is to be measured)
#   max_error   (The maximum error of the IL calculation. Error is relative to actual values.)
# Some other available arguments are:
#   lane_factor (The lane_factor. Can be a list for different factors for each lane, or a singular value for same lane_factor for all. default 1.0)
#   POT         (Minimum LE to be included in Peak Over Threshold (POT) analysis. default to 0.0)
bridge.add_load_effect(load_effect = "M", poi = 3/4 * L[0], max_error = 0.001)

# Defining the lane flow
# Lane flow can be constructed using:
#   - data frame (NOT recommended as it is prone to error)
#   - csv file
#   - default files
# Construction via data frame is done using the Pandas.DataFrame arguments and syntax. See Pandas documentation for details.
# Construction via csv file require you to supply the following arguments:
#   path        (Path to the csv file, in string)
# Construction via the default files can be done using the load_default() method. The method can take the argument:
#   sample      (The sample number, either 1 or 2. Currently there are two default lane flow definitions available. Default to 1)
lane_1 = LaneFlow()
lane_1.load_default()


# Define the Bridge Flows
# Multiple lanes can be created. This is then added to a BridgeFlow object, which describe the flow of all the lanes in the bridge
# Lane can be added at initialisation as list or BtlsFlow object, or after initialisation via:
#   - add() method or other aliases of the method: add_lane_flow(), append(), extend())
#   - insert(idx, lane_flow) : insert new lane flow into the array
#   - list indexing          : This is similar to regular list indexing, e.g. bridge_flow[1] = lane_2, del bridge_flow[0], etc.
bridge_flow = BridgeFlow(lane_1)

In [6]:
# DEVELOPMENT VERSION
# THIS WILL CHANGE IN THE FUTURE AS THE txt FILES ARE DEPRECATED

# Creating BTLS wrapper
# The bridge needs to then be wrapped in a BTLSWrapper class. This class provides the method of parsing the required input and reading outputs.
# Arguments here are:
# bridge        (A Bridge object)
# num_days      (Number of days for the simulation)
# working_dir   (The working directory where all txt files will be created and read)
btls = BTLSWrapper(bridge = bridge, bridge_flow = bridge_flow, num_days = 100)

# To change analysis types and other options, see the options object
# Set the Simulation mods:
btls.options.program_mode = "Gen & Sim"
# In this case, we're interested in block maxima analysis. We could also do interval statistics to get a quick overview later on
# We can invoke the setiings one by one, e.g.:
#btls.options.analyse_block_max = 1
#btls.options.write_block_max_summary_files = 1
#btls.options.write_block_max_mixed_analysis = 1
# A simpler way to do this is for a set of analysis, you could do:
btls.options.group_analyse_block_maxima = True
btls.options.group_analyse_statistics = True
# Or to turn off an analysis (by default all analysis are off)
btls.options.group_analyse_pot = False

#Other options, e.g. set simulation time step
btls.options.time_step = 0.01
btls.options.write_vehicle_file = 1
btls.options.write_vehicle_file_format = "MON"

# Define the traffic objects
# NEED WORK HERE
#Traffic Vehicle Settings
btls.options.vehicle_generation_model = "Constant"
btls.options.headway_model = "Free Flow"

constant_vehicle = ConstantVehicle()
constant_vehicle.load_default()
btls.constant_vehicle = constant_vehicle

garage = Garage()
garage.load_default()
btls.garage = garage

kernels = Kernels()
kernels.load_default()
btls.kernels = kernels

btls.traffic = Auxerre #OLD CODE, BUT NO REPLACEMENT YET

# We can then run the analysis. Output will be read automatically
btls.run()

# SAVING AND LOADING
# Currently not working. Its not saving some attributes correctly, e.g. btls.outputs.block_maxima_summary[0].descriptor
# Potentialr ref:
# https://stackoverflow.com/questions/35783898/python-pickle-not-saving-entire-object
# https://stackoverflow.com/questions/18361678/why-cant-pickle-a-list-that-is-an-attribute-of-an-object
#fname = 'Example 1 Bridge Block Maxima Analysis.obj'
#btls.save(fname)
#
#btls = BTLSWrapper
#btls = btls.load(fname)

---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...


*** Warning: Influence Surface file could not be opened


1	2	3	4	5	6	7	8	9	10
11	12	13	14	15	16	17	18	19	20
21	22	23	24	25	26	27	28	29	30
31	32	33	34	35	36	37	38	39	40
41	42	43	44	45	46	47	48	49	50
51	52	53	54	55	56	57	58	59	60
61	62	63	64	65	66	67	
Flushing buffer of 1000000 vehicles at 18/3/0 14:26:21.6795
68	69	70
71	72	73	74	75	76	77	78	79	80
81	82	83	84	85	86	87	88	89	90
91	92	93	94	95	96	97	98	99	100


Flushing buffer of 479697 vehicles at 1/5/0 0:0:5.61722

Simulation complete

Duration of analysis: 32.901 s


# Analysis
The output of the analysis is store in btls.output object For example, if we're interested in the block maxima seperated by number of vehicles in a crossing:

In [7]:
#display function is not needed if you want to display only one output for one cell. Its use here so we don't need multiple cells.
display(btls.outputs.block_maxima_separated_vehicles[3])
display(btls.outputs.block_maxima_separated_vehicles[1])
display(btls.outputs.block_maxima_separated_vehicles[0])
display(btls.outputs.block_maxima_separated_vehicles[2])

,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,2099.9,59981.3,20.50,1,1 vehicle(s) Vehicle object at 0x7f87190b5330
1,1,2,1586.0,6234.3,21.74,1,1 vehicle(s) Vehicle object at 0x7f87190b53f0
2,2,1,2099.9,94376.4,20.50,1,1 vehicle(s) Vehicle object at 0x7f87190b5450
3,2,2,1586.0,87149.1,21.74,1,1 vehicle(s) Vehicle object at 0x7f87190b5540
4,3,1,2099.9,204799.1,20.50,1,1 vehicle(s) Vehicle object at 0x7f87190b5600
...,...,...,...,...,...,...,...
195,98,2,1586.0,8440654.7,21.74,1,1 vehicle(s) Vehicle object at 0x7f87190d65c0
196,99,1,2099.9,8511608.6,20.50,1,1 vehicle(s) Vehicle object at 0x7f87190d6680
197,99,2,1586.0,8490891.5,21.74,1,1 vehicle(s) Vehicle object at 0x7f87190d6740
198,100,1,2099.9,8604791.7,20.50,1,1 vehicle(s) Vehicle object at 0x7f87190d6800


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,153.4,86104.8,28.91,4,4 vehicle(s) Vehicle object at 0x7f871b2bfa60
1,1,2,124.0,86104.8,27.95,4,4 vehicle(s) Vehicle object at 0x7f871b2bfb50
2,4,1,158.8,281000.8,28.95,4,4 vehicle(s) Vehicle object at 0x7f871b2bf940
3,4,2,131.4,281000.7,27.00,4,4 vehicle(s) Vehicle object at 0x7f871b2bf700
4,10,1,170.5,809889.1,27.82,4,4 vehicle(s) Vehicle object at 0x7f871b2bf970
5,10,2,129.4,809889.1,28.61,4,4 vehicle(s) Vehicle object at 0x7f871b2bfd60
6,33,1,150.1,2789819.4,28.76,4,4 vehicle(s) Vehicle object at 0x7f871b2bfe50
7,33,2,118.8,2789819.4,28.76,4,4 vehicle(s) Vehicle object at 0x7f871b2bf6a0
8,43,1,166.8,3713059.5,28.08,4,4 vehicle(s) Vehicle object at 0x7f87704ced70
9,43,2,129.2,3713059.5,28.33,4,4 vehicle(s) Vehicle object at 0x7f8770ec8730


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,2156.0,76093.7,32.92,3,3 vehicle(s) Vehicle object at 0x7f871b623850
1,1,2,1507.6,76093.7,32.92,3,3 vehicle(s) Vehicle object at 0x7f871b621ae0
2,2,1,2078.0,154265.4,28.78,3,3 vehicle(s) Vehicle object at 0x7f871b6219f0
3,2,2,1470.6,169133.5,23.59,3,3 vehicle(s) Vehicle object at 0x7f871b621ab0
4,3,1,2055.1,258154.6,28.79,3,3 vehicle(s) Vehicle object at 0x7f871b623790
...,...,...,...,...,...,...,...
195,98,2,1656.5,8449557.9,31.06,3,3 vehicle(s) Vehicle object at 0x7f871b57bd30
196,99,1,1956.9,8506055.6,28.92,3,3 vehicle(s) Vehicle object at 0x7f871b57bdf0
197,99,2,1456.4,8520142.6,23.75,3,3 vehicle(s) Vehicle object at 0x7f871b57beb0
198,100,1,2134.5,8584439.5,28.79,3,3 vehicle(s) Vehicle object at 0x7f871b57bf70


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,2546.5,13190.1,21.68,2,2 vehicle(s) Vehicle object at 0x7f8715ecf250
1,1,2,1983.5,13190.3,26.79,2,2 vehicle(s) Vehicle object at 0x7f8715ecf310
2,2,1,2522.5,159753.3,21.66,2,2 vehicle(s) Vehicle object at 0x7f8715ecf370
3,2,2,1962.4,159753.5,26.75,2,2 vehicle(s) Vehicle object at 0x7f8715ecf460
4,3,1,2565.0,215899.5,21.55,2,2 vehicle(s) Vehicle object at 0x7f8715ecf520
...,...,...,...,...,...,...,...
195,98,2,2007.4,8388085.0,26.76,2,2 vehicle(s) Vehicle object at 0x7f8715e54520
196,99,1,2562.4,8506055.8,21.71,2,2 vehicle(s) Vehicle object at 0x7f8715e545e0
197,99,2,2006.3,8506056.0,26.84,2,2 vehicle(s) Vehicle object at 0x7f8715e546a0
198,100,1,2500.9,8597593.5,21.65,2,2 vehicle(s) Vehicle object at 0x7f8715e54760


Suppose we want to see:  
for a 3 vehicles crossing event, the vehicles that creates the most extreme load_effect for Load Effect 1. We want to see their headway at the maximum loading event.  
We can use regular pandas expressions and methods.

In [8]:
# Shorthand for the output object we're interested in
df = btls.outputs.block_maxima_separated_vehicles[0]
# Filter only load_effect 1
df[df['load_effect_num'] == 1]
# Get the critical vehicles
critical_platoon = df["vehicles"][np.argmax(df.max_load_effect)]
#Calculate their headways
arrival_time_in_second = (critical_platoon.second + critical_platoon.minute * 60 + critical_platoon.hour * 3600).to_numpy()
headway = arrival_time_in_second[1:] - arrival_time_in_second[0:-1]

# Display the results
print("Headways of the critical vehicles for 3 vehicles crossing event are:", headway, "seconds")
display(critical_platoon)

Headways of the critical vehicles for 3 vehicles crossing event are: [0.548 0.598] seconds


,head,day,month,year,hour,minute,second,num_axle,num_axle_groups,gvw,...,AW11,AS11,AW12,AS12,AW13,AS13,AW14,AS14,AW15,AS15
0,1001,1,3,2010,0,13,35.569,5,0,45872.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,1,3,2010,0,13,36.117,5,0,45872.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001,1,3,2010,0,13,36.715,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Parallelisation Module
For faster calculations, BTLS can utilise multiple cores to do the simulations in parallel. This is done using BTLSParallelWrappper Object.  
We will simulate 1 year of data for 6 bridges of varying spans.

In [10]:
from PyBTLS.py_module.parallel.parallel import BTLSParallelWrapper

In [12]:
span_array = np.array([10.0, 15.0, 20.0, 25.0, 30.0, 35.0])
btls_parallel = BTLSParallelWrapper(num_subprocess = 6)
for i, span in enumerate(span_array):
    L = [span]
    R = [-1, 0, -1, 0] #Simply supported
    bridge = Bridge(L=L, R=R, n_lane=1)

    bridge.add_load_effect(load_effect = 1) 
    bridge.add_load_effect(load_effect = "M", poi = 3/4 * L[0], max_error = 0.001)

    lane_1 = LaneFlow()
    lane_1.load_default()

    bridge_flow = BridgeFlow(lane_1)

    btls = BTLSWrapper(bridge = bridge, bridge_flow = bridge_flow, num_days = 5)

    btls.options.program_mode = "Gen & Sim"
    btls.options.group_analyse_block_maxima = True
    btls.options.group_analyse_statistics = True
    btls.options.group_analyse_pot = False
    btls.options.time_step = 0.01
    btls.options.write_vehicle_file = 1
    btls.options.write_vehicle_file_format = "MON"
    btls.options.vehicle_generation_model = "Grave"
    btls.options.headway_model = "Free Flow"
    constant_vehicle = ConstantVehicle()
    constant_vehicle.load_default()
    btls.constant_vehicle = constant_vehicle
    garage = Garage()
    garage.load_default()
    btls.garage = garage
    kernels = Kernels()
    kernels.load_default()
    btls.kernels = kernels
    btls.traffic = Auxerre #OLD CODE, BUT NO REPLACEMENT YET


    # Instead of running, we add it to the BTLSParallelWrapper object
    btls_parallel.add(btls)


In [13]:
#Run it!
btls_parallel.run();

Starting Kernel 0


/home/default/Documents/btls/BtlsParallel/btls/py_src/PyBTLS/py_module/parallel/wrapper.py:55: UserWarning: Working directory already existed, and so was deleted
  warnings.warn("Working directory already existed, and so was deleted")


Starting Kernel 1
---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...


*** Warning: Influence Surface file could not be opened


1	2	3	4	5	

Flushing buffer of 74401 vehicles at 6/1/0 0:0:0.235136
Starting Kernel 2

Simulation complete

Duration of analysis: 1.627 s
---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...


...

1	2	3	4	5	

Flushing buffer of 74328 vehicles at 6/1/0 0:0:0.186161

Simulation complete

Duration of analysis: 1.959 s
Starting Kernel 5
---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...


*** Warning: Influence Surface file could not be opened


1	2	3	4	5	

Flushing buffer of 74133 vehicles at 6/1/0 0:0:2.11704

Simulation complete

Duration of analysis: 2.576 s
---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...


*** Warning: Influence Surface file could not be opened


1	2	3	4	5	

Flushing buffer of 74281 vehicles at 6/1/0 0:0:1.84878

Simulation complete

Duration of analysis: 2.289 s


In [14]:
#See the outputs!
display(btls_parallel[0].outputs.block_maxima_summary)
display(btls_parallel[0].outputs.block_maxima_summary[0])

[Load effect 0:  5 row(s) of BlockMaximaSummary object at 0x7f871b660ca0,
 Load effect 1:  5 row(s) of BlockMaximaSummary object at 0x7f871b661300]

,index,event_1_truck,event_2_truck
0,1,740.0,656.2
1,2,822.5,583.1
2,3,764.6,679.4
3,4,802.2,624.7
4,5,778.8,649.8


In [15]:
#Example analysis: Print the block max summary of load effect 1 for all the bridges
load_effect_num = 1
for b in btls_parallel:
    print("=============")
    print("Span =", b.bridge.L, "m")
    display(b.outputs.block_maxima_summary[load_effect_num])

Span = [10.0] m


,index,event_1_truck,event_2_truck
0,1,557.0,543.5
1,2,622.7,480.9
2,3,580.8,530.1
3,4,603.2,499.4
4,5,576.0,503.8


Span = [15.0] m


,index,event_1_truck,event_2_truck,event_3_truck
0,1,1002.9,860.2,72.9
1,2,952.8,883.3,50.7
2,3,1108.6,888.6,78.9
3,4,977.8,832.2,48.3
4,5,1199.5,936.4,292.2


Span = [20.0] m


...

,index,event_1_truck,event_2_truck,event_3_truck,event_4_truck
0,1,2288.1,2031.6,1274.3,0.0
1,2,2162.2,2402.4,1534.8,0.0
2,3,2200.5,2122.6,1514.2,0.0
3,4,2129.2,1994.2,1344.1,0.0
4,5,2202.8,2202.9,1373.7,116.9


Span = [30.0] m


,index,event_1_truck,event_2_truck,event_3_truck,event_4_truck
0,1,2838.9,2858.1,3006.9,0.0
1,2,2811.0,2810.3,2457.4,0.0
2,3,2694.5,2728.2,2082.4,431.1
3,4,2679.9,2662.4,2060.3,0.0
4,5,2827.3,2789.3,2712.5,0.0


Span = [35.0] m


,index,event_1_truck,event_2_truck,event_3_truck,event_4_truck
0,1,3409.0,3335.5,2832.7,588.0
1,2,3728.8,3407.5,2912.8,858.9
2,3,3281.4,3724.0,3394.5,192.0
3,4,3235.1,3768.1,2833.8,227.3
4,5,3263.3,3573.5,2807.2,210.4


In [16]:
# DEVELOPMENT NOTES:
# Current available outputs are:

print("=============")
print("Interval statistics (List array):")
display(btls.outputs.interval_statistics)
print("=============")
print("Cumulative statistics:")
display(btls.outputs.cumulative_statistics)
print("=============")
print("Block maxima summary (List array):")
display(btls.outputs.block_maxima_summary)
print("=============")
print("Block maxima separated vehicles (List array):")
display(btls.outputs.block_maxima_separated_vehicles)
print("=============")
print("Block maxima mixed vehicles:")
display(btls.outputs.block_maxima_mixed_vehicles)
print("=============")
print("Generated vehicle file:")
display(btls.outputs.vehicles) #

Interval statistics (List array):


[Load effect 0:  4 row(s) of IntervalStatistics object at 0x7f8716ca5360,
 Load effect 1:  4 row(s) of IntervalStatistics object at 0x7f8719265630]

Cumulative statistics:


,load_effect,num_events,num_event_vehicles,num_event_trucks,min,max,mean,std_dev,variance,skeweness,kurtosis,event_1_truck,event_2_truck,event_3_truck
0,1,89622,106811,21393,0.14,4948.69,588.00,953.58,909306.45,2.03,2.72,20019.0,684.0,2.0
1,2,89622,106811,21393,0.21,3768.12,446.54,718.23,515859.03,2.05,2.81,20019.0,684.0,2.0


Block maxima summary (List array):


...

[10 x BlockMaximaEvent object at 0x7f8718fd88b0,
 10 x BlockMaximaEvent object at 0x7f8716c84910,
 10 x BlockMaximaEvent object at 0x7f8716c848e0,
 10 x BlockMaximaEvent object at 0x7f8716c84190]

Block maxima mixed vehicles:


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,4461.4,26238.2,26.18,1,1 vehicle(s) Vehicle object at 0x7f86cff040d0
1,1,2,3409.0,66701.0,29.33,1,1 vehicle(s) Vehicle object at 0x7f86cff073d0
2,2,1,4831.5,91379.3,26.71,1,1 vehicle(s) Vehicle object at 0x7f86cff042b0
3,2,2,3728.8,91379.4,29.31,1,1 vehicle(s) Vehicle object at 0x7f86cff04100
4,3,1,4658.8,174432.6,25.58,1,1 vehicle(s) Vehicle object at 0x7f86cff04040
5,3,2,3724.0,199305.1,34.10,2,2 vehicle(s) Vehicle object at 0x7f86cff05e10
6,4,1,4948.7,313557.9,33.48,2,2 vehicle(s) Vehicle object at 0x7f86cff05d20
7,4,2,3768.1,313558.1,36.70,2,2 vehicle(s) Vehicle object at 0x7f86cff06050
8,5,1,4698.4,377927.9,28.53,2,2 vehicle(s) Vehicle object at 0x7f86cff05c90
9,5,2,3573.5,377928.1,35.08,2,2 vehicle(s) Vehicle object at 0x7f86cff05db0


Generated vehicle file:


,head,day,month,year,hour,minute,second,num_axle,num_axle_groups,gvw,...,AW11,AS11,AW12,AS12,AW13,AS13,AW14,AS14,AW15,AS15
0,1001,1,1,2010,0,0,0.284,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,1,1,2010,0,0,1.437,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001,1,1,2010,0,0,5.607,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001,1,1,2010,0,0,11.967,4,0,40984.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001,1,1,2010,0,0,17.182,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74276,1001,5,1,2010,23,59,52.350,4,0,50831.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74277,1001,5,1,2010,23,59,55.020,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74278,1001,5,1,2010,23,59,56.045,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74279,1001,5,1,2010,23,59,58.898,4,0,39152.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
